# Training

In [4]:
from google.colab import userdata
roboflow_key = userdata.get('roboflow_key')

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=roboflow_key)
project = rf.workspace("smokefirev1").project("firesmoke-uendo")
version = project.version(2)
dataset = version.download("folder")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to firesmoke-2 in folder:: 100%|██████████| 9910/9910 [00:02<00:00, 3999.72it/s]


In [ ]:
%rm -rf /content/firesmoke-2/test/fire

In [ ]:
%rm -rf /content/firesmoke-2/train/fire

In [ ]:
import shutil
shutil.rmtree("/content/firesmoke-2/train/default fire smoke")

In [ ]:
import shutil
import os

root_dir = "/content/firesmoke-2/train"

def delete_checkpoints(root_dir):
    for root, dirs, files in os.walk(root_dir):
        if '.ipynb_checkpoints' in dirs:
            dir_path = os.path.join(root, '.ipynb_checkpoints')
            shutil.rmtree(dir_path)
            print(f"Deleted {dir_path}")

delete_checkpoints('.')


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Path to your dataset folder
train_path = '/content/firesmoke-2/train'
test_path = "/content/firesmoke-2/test"

# Load images and labels
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, (64, 64))  # Resize image to fixed size
                    images.append(img.flatten())  # Flatten the image
                    labels.append(label)
    return np.array(images), np.array(labels)

# Load dataset
X_train, y_train = load_images_from_folder(train_path)
X_test, y_test = load_images_from_folder(test_path)

X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Encode labels to integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the Decision Tree Classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.74


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

accuracy = f1_score(y_test, y_pred)
print(f'F1: {accuracy:.2f}')

accuracy = precision_score(y_test, y_pred)
print(f'Precision: {accuracy:.2f}')

accuracy = recall_score(y_test, y_pred)
print(f'Recall: {accuracy:.2f}')

Accuracy: 0.74
F1: 0.78
Precision: 0.79
Recall: 0.77


In [ ]:
import urllib.request
from PIL import Image

url = "https://cdn.medcom.id/dynamic/content/2020/09/29/1193467/aUGWkuAJen.jpeg?w=1024"
urllib.request.urlretrieve(url, "noasap1.png")

url = "https://th.bing.com/th/id/OIP.qsYlpLu-jjaBS5FWT4Y-dQAAAA?rs=1&pid=ImgDetMain"
urllib.request.urlretrieve(url, "asap1.png")

url = "https://awsimages.detik.net.id/community/media/visual/2022/07/28/pkt-pastikan-kondisi-kondusif-pasca-pabrik-alami-over-firing-1_169.jpeg?w=650&q=80"
urllib.request.urlretrieve(url, "noasap2.jpeg")

('noasap2.jpeg', <http.client.HTTPMessage at 0x7e55c34098a0>)

In [ ]:
def load_images(img_path, label):
    images = []
    labels = []
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
      img = cv2.resize(img, (64, 64))  # Resize image to fixed size
      images.append(img.flatten())  # Flatten the image
      labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
image, label = load_images("/content/asap1.png", "smoke")
label = label_encoder.transform(label)
image = scaler.transform(image)
pred = clf.predict(image)
print(label_encoder.inverse_transform(pred))

['smoke']


In [ ]:
image, label = load_images("/content/noasap1.png", "default")
label = label_encoder.transform(label)
image = scaler.transform(image)
pred = clf.predict(image)
print(label_encoder.inverse_transform(pred))

['default']


In [ ]:
image, label = load_images("//content/noasap2.jpeg", "default")
label = label_encoder.transform(label)
image = scaler.transform(image)
pred = clf.predict(image)
print(label_encoder.inverse_transform(pred))

['default']


In [ ]:
import pickle

# save
with open('model.pkl','wb') as f:
    pickle.dump(clf,f)

In [ ]:
# save
with open('image_scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

with open('label_encoder.pkl','wb') as f:
    pickle.dump(label_encoder,f)

In [ ]:
# Create a parameter grid
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a decision tree classifier
clf_default = DecisionTreeClassifier()

# Perform grid search
grid_search = GridSearchCV(clf_default, param_grid, cv=5)
grid_search.fit(X_train, y_train)


# Print the best parameters
print(grid_search.best_params_)

{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}


In [ ]:
# Initialize and train the Decision Tree Classifier
clf_tuning = DecisionTreeClassifier(max_depth= 5, min_samples_leaf=4, min_samples_split=10)
clf_tuning.fit(X_train, y_train)

# Make predictions
y_pred = clf_tuning.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

accuracy = f1_score(y_test, y_pred)
print(f'F1: {accuracy:.2f}')

accuracy = precision_score(y_test, y_pred)
print(f'Precision: {accuracy:.2f}')

accuracy = recall_score(y_test, y_pred)
print(f'Recall: {accuracy:.2f}')

Accuracy: 0.78
F1: 0.82
Precision: 0.80
Recall: 0.84


In [ ]:
import pickle

# save
with open('model_tuning.pkl','wb') as f:
    pickle.dump(clf_tuning,f)

In [ ]:
image, label = load_images("/content/asap1.png", "smoke")
label = label_encoder.transform(label)
image = scaler.transform(image)
pred = clf_tuning.predict(image)
print(label_encoder.inverse_transform(pred))

['smoke']


In [ ]:
image, label = load_images("/content/noasap1.png", "default")
label = label_encoder.transform(label)
image = scaler.transform(image)
pred = clf_tuning.predict(image)
print(label_encoder.inverse_transform(pred))

['smoke']


In [ ]:
image, label = load_images("//content/noasap2.jpeg", "default")
label = label_encoder.transform(label)
image = scaler.transform(image)
pred = clf_tuning.predict(image)
print(label_encoder.inverse_transform(pred))

['default']


# Visualization

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

In [ ]:
import pickle

clf = pickle.load(open("/content/model_tuning.pkl", "rb"))

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=[str(i) for i in range(X_train.shape[1])],
                                filled=True, rounded=True,
                                special_characters=True)
graph = graphviz.Source(dot_data)
graph.render("decision_tree_image")

'decision_tree_image.pdf'